In [1]:
import zipfile
from matplotlib import pyplot as plt
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [9]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
!unzip crawl-300d-2M.vec.zip

--2023-10-22 03:39:36--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.96, 3.163.189.14, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1523785255 (1.4G) [application/zip]
Saving to: ‘crawl-300d-2M.vec.zip’

crawl-300d-2M.vec.z 100%[===================>]   1.42G   216MB/s    in 8.9s    

2023-10-22 03:39:45 (163 MB/s) - ‘crawl-300d-2M.vec.zip’ saved [1523785255/1523785255]

Archive:  crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       


In [12]:
!wget -O Sherlock.txt https://www.gutenberg.org/ebooks/48320.txt.utf-8

--2023-10-22 03:42:49--  https://www.gutenberg.org/ebooks/48320.txt.utf-8
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-22 03:42:49--  http://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/cache/epub/48320/pg48320.txt [following]
--2023-10-22 03:42:49--  https://www.gutenberg.org/cache/epub/48320/pg48320.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622314 (608K) [text/plain]
Saving to: ‘Sherlock.txt’

Sherlock.txt        100%[===================>] 607.73

In [2]:
with open("Sherlock.txt", 'r', encoding='utf-8') as file:
    book = file.read()

In [3]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr|no|No)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = ['<SOS> ' + s.strip() + ' <EOS>' for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

In [4]:
book = split_into_sentences(book)

In [5]:
book[1000:1010]

['<SOS> I had come to the conclusion that he had dropped asleep, and indeed was nodding myself, when he suddenly sprang out of his chair with the gesture of a man who has made up his mind, and put his pipe down upon the mantel-piece. <EOS>',
 '<SOS> “Sarasate plays at the St. James’s Hall this afternoon,” he remarked. <EOS>',
 '<SOS> “What do you think, Watson? <EOS>',
 '<SOS> Could your patients spare you for a few hours? <EOS>',
 '<SOS> ”  “I have nothing to do to-day. <EOS>',
 '<SOS> My practice is never very absorbing”. <EOS>',
 '<SOS> “Then put on your hat and come. <EOS>',
 '<SOS> I am going through the city first, and we can have some lunch on the way. <EOS>',
 '<SOS> I observe that there is a good deal of German music on the programme, which is rather more to my taste than Italian or French. <EOS>',
 '<SOS> It is introspective, and I want to introspect. <EOS>']

In [110]:
token = Tokenizer(num_words=1000,
                  filters='!"“”#$%&()*+,.-/:;=?@[\\]^_`{|}~\t\n\ufeff\u2002', lower=True,
                  split=' ', char_level=False, oov_token="UNK")
token.fit_on_texts(book)

In [111]:
sequences = token.texts_to_sequences(book)

In [112]:
textSequences = token.sequences_to_texts(sequences)

In [113]:
maxLen = len(max(sequences, key=len))

In [114]:
def create_dataset(sequences, maxSetPerSentence=3):
    inputs = []
    outputs = []
    for i, sent in enumerate(sequences):
        if len(sent) <= maxSetPerSentence+2:
          continue

        samples = np.random.choice(range(2, len(sent)), maxSetPerSentence, replace=False)
        for j in samples:
          inputs.append(sent[:j])
          outputs.append(sent[j])

    return inputs, outputs

In [115]:
X, y = create_dataset(sequences)

In [108]:
X[1]

[2, 4, 147, 308, 880, 7, 881, 7, 124, 37, 30]

In [109]:
y[1]

880

In [116]:
maxLen = len(max(X, key=len))
train_sequences = pad_sequences(X, maxlen=maxLen, padding="post")
train_sequences_y = np.array(y)
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])

In [15]:
import codecs
import IPython.display as ipd
from IPython.display import Audio, update_display
from ipywidgets import IntProgress

#load embeddings
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('./crawl-300d-2M.vec', encoding='utf-8')
N = 2000000
bar = IntProgress(min=0, max=N)
ipd.display(bar)

i = 0
for n, line in enumerate(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    if n//(N//100) > i:
          bar.value = n
          i += 1
f.close()
print('found %s word vectors' % len(embeddings_index))

loading word embeddings...


IntProgress(value=0, max=2000000)

found 1999996 word vectors


In [133]:
import pickle

a_file = open("data.pkl", "wb")
pickle.dump(embeddings_index, a_file)
a_file.close()


In [117]:
embed_dim=300
num_words=len(dictionary)+1
embedding_matrix=np.zeros([num_words,embed_dim])
for word, idx in dictionary.items():
  if word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [118]:
x_train, x_test, y_train, y_test = train_test_split(train_sequences, train_sequences_y, test_size=0.3)

In [183]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input, Concatenate, Dot, RepeatVector, TimeDistributed, Multiply, Lambda, Flatten, Activation, Reshape, BatchNormalization, LSTM
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau

def softMaxOverTime(x):
    return softmax(x, axis=0)

In [188]:
value_dim = 100 #Tamaño de los estados del contextualizador
nb_words = num_words #Vocabulary size

input_layer = Input(shape=(maxLen,)) #Cantidad maxima de la frase de entrada, estos son los timesteps para este caso
embedding_layer = Embedding(nb_words, embed_dim, weights=[embedding_matrix], input_length=maxLen, trainable=False)(input_layer)

#Contextualizador
lstm_out = LSTM(value_dim, return_sequences=True, activation="tanh")(embedding_layer)
ulog_attention = Dense(1, activation="linear")(lstm_out)

#Attention
attention = Activation(softMaxOverTime)(ulog_attention)
repeated_attention = TimeDistributed(RepeatVector(value_dim))(attention)
repeated_attention = Reshape([maxLen, value_dim])(repeated_attention)

#Pesamos el embedding con el score
weighted_embeddings = Multiply()([repeated_attention, lstm_out])
embedding_sum = Lambda(lambda x: K.sum(x, axis=1))(weighted_embeddings)

#Salida
dense1 = Dense(100, activation='relu')(embedding_sum)
dense2 = Dense(100, activation='relu')(dense1)
dense3 = Dense(nb_words, activation='softmax')(dense2)
model = Model(input_layer , dense3)

In [185]:
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 97)]              0         
                                                                 
 embedding_10 (Embedding)    (None, 97, 300)           2653200   
                                                                 
 lstm_10 (LSTM)              (None, 97, 100)           160400    
                                                                 
 dense_31 (Dense)            (None, 97, 100)           10100     
                                                                 
 dense_32 (Dense)            (None, 97, 100)           10100     
                                                                 
 dense_33 (Dense)            (None, 97, 8844)          893244    
                                                                 
Total params: 3727044 (14.22 MB)
Trainable params: 1073844

In [189]:
opt = optimizers.Adam(learning_rate=0.5)
callbackROP = ReduceLROnPlateau(monitor="loss", factor=0.5, patience=10)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(x=x_train, y=y_train, validation_data=(x_test, y_test), batch_size=256, epochs=50, callbacks=[callbackROP])

Epoch 1/50


In [139]:
toNum = token.texts_to_sequences(["<SOS> Hello"])

In [156]:
aver = token.sequences_to_texts([[2]])

In [135]:
toNum = pad_sequences(toNum, maxlen=maxLen, padding="post")

In [142]:
out = model.predict(x_train)

407/407 [==============================] - 3s 8ms/step
